In [1]:
!wget https://raw.githubusercontent.com/AJV009/Machine_Learning_NB/master/datasets/industry_energy/energy_train.csv

--2020-04-30 13:10:29--  https://raw.githubusercontent.com/AJV009/Machine_Learning_NB/master/datasets/industry_energy/energy_train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4375067 (4.2M) [text/plain]
Saving to: ‘energy_train.csv’

energy_train.csv    100%[===================>]   4.17M  --.-KB/s    in 0.06s   

2020-04-30 13:10:30 (68.9 MB/s) - ‘energy_train.csv’ saved [4375067/4375067]



In [7]:
!wget https://raw.githubusercontent.com/AJV009/Machine_Learning_NB/master/datasets/industry_energy/energy_test.csv

--2020-04-30 13:16:22--  https://raw.githubusercontent.com/AJV009/Machine_Learning_NB/master/datasets/industry_energy/energy_test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1276609 (1.2M) [text/plain]
Saving to: ‘energy_test.csv’

energy_test.csv     100%[===================>]   1.22M  --.-KB/s    in 0.05s   

2020-04-30 13:16:22 (23.0 MB/s) - ‘energy_test.csv’ saved [1276609/1276609]



In [0]:
import pandas as pd
energy_train_df = pd.read_csv('energy_train.csv')
energy_test_df = pd.read_csv('energy_test.csv')

In [0]:
energy_train_df["degree_C1"] = energy_train_df["degree_C1"].fillna(21.163095)
energy_train_df["degree_C3"] = energy_train_df["degree_C3"].fillna(21.566924)
energy_train_df["moisture_9"] = energy_train_df["moisture_9"].fillna(41.646994)
energy_test_df["moisture_9"] = energy_test_df["moisture_9"].fillna(41.052105)

In [0]:
energy_train_df["date"] = pd.to_datetime(energy_train_df["date"])

energy_train_df["year"] = energy_train_df["date"].dt.year
energy_train_df["month"] = energy_train_df["date"].dt.month
#energy_train_df["week"] = energy_train_df["date"].dt.week
energy_train_df["day"] = energy_train_df["date"].dt.day
energy_train_df["hour"] = energy_train_df["date"].dt.hour
energy_train_df["minute"] = energy_train_df["date"].dt.minute
#energy_train_df["second"] = energy_train_df["date"].dt.second

energy_test_df["date"] = pd.to_datetime(energy_test_df["date"])
#energy_test_df = energy_test_df.drop(columns = ["date"])
energy_test_df["year"] = energy_test_df["date"].dt.year
energy_test_df["month"] = energy_test_df["date"].dt.month
#energy_test_df["week"] = energy_test_df["date"].dt.week
energy_test_df["day"] = energy_test_df["date"].dt.day
energy_test_df["hour"] = energy_test_df["date"].dt.hour
energy_test_df["minute"] = energy_test_df["date"].dt.minute

energy_test_df = energy_test_df.drop(columns = ["id","date"])
energy_train_df = energy_train_df.drop(columns = "id")

In [0]:
for i in range(1,10):
    energy_train_df['EMA_C'+str(i)] = energy_train_df['degree_C'+str(i)].ewm(span=10,adjust=False).mean()
    energy_train_df['EMA_M'+str(i)] = energy_train_df['moisture_'+str(i)].ewm(span=10,adjust=False).mean()

energy_train_df['EMA_degree_Cout'] = energy_train_df["degree_Cout"].ewm(span=10,adjust=False).mean()
energy_train_df['EMA_Pressure'] = energy_train_df["Pressure"].ewm(span=10,adjust=False).mean()
energy_train_df['EMA_moisture_out'] = energy_train_df["moisture_out"].ewm(span=10,adjust=False).mean()
energy_train_df['EMA_Wind'] = energy_train_df["Wind"].ewm(span=10,adjust=False).mean()
energy_train_df['EMA_Clarity'] = energy_train_df["Clarity"].ewm(span=10,adjust=False).mean()
energy_train_df['EMA_dew_index'] = energy_train_df["dew_index"].ewm(span=10,adjust=False).mean()

In [0]:
energy_train_df= energy_train_df.drop(columns = ["moisture_6","EMA_M6"])

In [0]:
for i in range(1,10):
    energy_test_df['EMA_C'+str(i)] = energy_test_df['degree_C'+str(i)].ewm(span=10,adjust=False).mean()
    energy_test_df['EMA_M'+str(i)] = energy_test_df['moisture_'+str(i)].ewm(span=10,adjust=False).mean()

energy_test_df['EMA_degree_Cout'] = energy_test_df["degree_Cout"].ewm(span=10,adjust=False).mean()
energy_test_df['EMA_Pressure'] = energy_test_df["Pressure"].ewm(span=10,adjust=False).mean()
energy_test_df['EMA_moisture_out'] = energy_test_df["moisture_out"].ewm(span=10,adjust=False).mean()
energy_test_df['EMA_Wind'] = energy_test_df["Wind"].ewm(span=10,adjust=False).mean()
energy_test_df['EMA_Clarity'] = energy_test_df["Clarity"].ewm(span=10,adjust=False).mean()
energy_test_df['EMA_dew_index'] = energy_test_df["dew_index"].ewm(span=10,adjust=False).mean()

energy_test_df= energy_test_df.drop(columns = ["moisture_6","EMA_M6"])

In [14]:
print("Shape of train: "+str(energy_train_df.shape))
print("Shape of test: "+str(energy_test_df.shape))

Shape of train: (15000, 56)
Shape of test: (4375, 54)


In [0]:
y_train = energy_train_df[['WattHour']].copy()
X_train = energy_train_df.drop(columns=["WattHour","date"])

In [16]:
print(X_train.shape)
print(y_train.shape)
print(energy_test_df.shape)

(15000, 54)
(15000, 1)
(4375, 54)


In [0]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

In [0]:
sc_X = StandardScaler()

X_train2 = sc_X.fit_transform(X_train)

X_test2 = sc_X.fit_transform(energy_test_df)

#sc_y = StandardScaler()
#sc_y.fit(y_train)
#y_train = sc_y.transform(y_train)

In [19]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

polynomial_features= PolynomialFeatures(degree=2)

polynomial_features2= PolynomialFeatures(degree=2)

x_poly = polynomial_features.fit_transform(X_train2)

test_poly= polynomial_features2.fit_transform(X_test2)

polynomial_features.fit(x_poly, y_train)

#x_poly, X_validate, y_train, y_validate = train_test_split(x_poly, y_train, test_size=0.05, random_state=1)

PolynomialFeatures(degree=2, include_bias=True, interaction_only=False,
                   order='C')

In [0]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()

In [21]:
reg.fit(x_poly,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
predict = reg.predict(test_poly)

In [0]:
output = pd.DataFrame(predict,columns = ["WattHour"])

In [25]:
output

,WattHour
0,143.636650
1,216.856197
2,263.235375
3,263.408025
4,279.255804
...,...
4370,179.652407
4371,140.250285
4372,104.681399
4373,107.114392
